# Meta-Prompting Scaffolding Tutorial

This Jupyter Notebook provides a hands-on tutorial on the concept of meta-prompting scaffolding, a method designed to enhance the efficiency of Language Models by using task-agnostic scaffolding strategies. The goal here is to demonstrate how we can enhance a standard language model by integrating specialized expert systems, resulting in an overall richer and more capable AI solution.

## Introduction

Meta-prompting is a sophisticated mechanism that enhances the language model's capability to handle complex tasks by breaking them down and distributing them among multiple expert models. Each expert tackles a portion of the task, and their insights are aggregated to provide a final, integrated solution.

This tutorial aims to implement a simplified version of meta-prompting scaffolding using OpenAI's Chat Completion API as our language model. 


## Pre-requisites

Before we start, ensure you have the `openai` Python package installed:

In [1]:
%%capture
!pip install openai

## Imports

We'll begin by importing the necessary libraries:

In [17]:
import openai
from openai import OpenAI
import json
from typing import List, Dict

## API Key Setup
Set your OpenAI API key as a string variable below. Remember not to expose your API keys in publicly accessible places.

In [18]:
OPENAI_API_KEY = 'INSERT_API_KEY'

client = OpenAI(api_key=OPENAI_API_KEY)

## Defining the Meta-Prompting Method
Next, we'll define a meta-prompting function that formulates the prompt, calls the language model, and processes the response.

### First a simplified method to test:

In [25]:
def simplified_meta_prompting(meta_prompt: str, 
                              prompt: str, 
                              task_description: str, 
                              language_model: str):
    """
    This function iteratively interacts with a specified language model by first sending
    a meta prompt and then refining the process with an expert model, if necessary.

    The process is as follows:
    1. The function sends a meta prompt to the language model, alongside the task description and an initial prompt.
    2. It then captures the initial response from the model.
    3. Assuming an expert model is necessary, it sends the initial response as input to an expert model conversation.
    4. The response from the expert model is captured and combined with the initial response.

    This simulates an iterative refinement of the model's response, potentially leveraging specialized models.

    Args:
    - meta_prompt (str): The initial role play or meta-contextual prompt that sets the conversation's context.
    - prompt (str): The initial user prompt to the language model, detailing what is expected in the conversation.
    - task_description (str): A description of the task at hand, which provides context for the question.
    - language_model (str): The identifier for the language model to be used in both initial and expert model interactions.

    Returns:
    - str: The final combined output from both the initial interaction and the expert model, providing a refined answer.
    """
    # Call the language model with the initial prompt
    response = client.chat.completions.create(
        model=language_model,
        messages=[
            {"role": "system", "content": meta_prompt},
            {"role": "user", "content": f"Question: {task_description}\n\n{prompt}"}
        ]
    )
    
    output = response.choices[0].message.content
    print("Initial Meta-Model Response:", output, "\n")
    
    # Here we would analyze the response to determine if any expert models need to be called
    # For this example, we will assume an expert model is needed and directly invoke it
    
    # Simulate calling an expert model
    expert_response = client.chat.completions.create(
        model=language_model,
        messages=[
            {"role": "system", "content": "Answer the question to the best of your abilities"},
            {"role": "user", "content": output}
        ],
        temperature=0.0
    )

    expert_output = expert_response.choices[0].message.content
    print("Expert Model Response:", expert_output, "\n")

    # Combine responses for the final output
    final_output = output + '\n' + expert_output
    return final_output


### Second a more complex method to test multiple interactions:

In [ ]:
def multi_round_meta_prompting(meta_prompt: str, 
                               prompt: str, 
                               task_description: str, 
                               language_model: str):
    """
    Facilitates a multi-round conversational interaction with a specified language model based on
    initial prompts and user or automated follow-up inputs. The conversation can evolve iteratively,
    allowing the model to refine its responses or provide additional information based on new inputs.

    The conversational loop continues until a stopping condition is met, such as the model generating
    a response flagged as a "FINAL ANSWER" or the user typing 'stop'.

    Args:
    - meta_prompt (str): A contextual setup for the language model that defines the role or persona it should adopt.
    - prompt (str): The initial prompt or question posed to the model.
    - task_description (str): Further explanation of the required task or the context surrounding the prompt.
    - language_model (str): Identifier for the language model to be used for generating responses.

    Returns:
    - list: A complete conversation history including system prompts, user inputs, and model responses. Each 
           conversation turn is represented as a dictionary with 'role' (either "system", "user", or "assistant") 
           and 'content' keys.
           
    """

    conversation = []
    

    conversation.append({
        "role": "system", 
        "content": meta_prompt
    })


    conversation.append({
        "role": "user", 
        "content": f"Question: {task_description}\n\n{prompt}"
    })

    while True:

        response = client.chat.completions.create(
            model=language_model,
            messages=conversation
        )


        model_response = response.choices[0].message.content
        conversation.append({
            "role": "assistant",
            "content": model_response
        })
        

#         print(f"Assistant: {model_response}")


        if ">> FINAL ANSWER:" in model_response:
            break
        

        # User might provide more input, or you might automate the continuation
        user_input = input("Your input (type 'stop' to end the conversation): ")
        if user_input.lower() == 'stop':
            break

        conversation.append({
            "role": "user",
            "content": user_input
        })

    return conversation

## Example Meta-Prompts

1. A simple meta-prompt where you are assiging your LLM agent a role

In [19]:
simple_meta_prompt = """Act as an expert Prompt Engineer.
I'll give you an initial prompt.\
Reason step by step to improve it.\
Write the final prompt as an elegant template with clear sections.\
Make sure you produce a ready-to-use prompt.\
"""

2. A multi agent meta-prompt where you ask your LLM agent to generate Expert agents that can collaborate to solve the question

In [6]:
multiagent_meta_prompt = """
When faced with a task, begin by identifying the participants who will contribute to solving the task. 
Provide profiles of the participants, describing their expertise or needs. 
Then, initiate a multi-round collaboration process until a final solution is reached. 
The participants will give critical comments and detailed suggestions whenever necessary.

Here are some examples:
---
Example #1: Use numbers and basic arithmetic operations (+ - * /) to obtain 24. 
You need to use all numbers, and each number can only be used once.
Input: 6 12 1 1

Participants: AI Assistant (you); Math Expert

Profiles: 
    - AI Assistant (you): A super-intelligent AI assistant capable of performing tasks more effectively than humans.
    - Math expert: A person who is good at math games, arithmetic calculation, and long-term planning.

Start collaboration!

Math Expert: Let's analyze the task in detail. You need to make sure that you meet the requirement, 
that you need to use exactly the four numbers (6 12 1 1) to construct 24. To reach 24, 
you can think of the common divisors of 24 such as 4, 6, 8, 3 and try to construct these first. 
Also you need to think of potential additions that can reach 24, such as 12 + 12.

AI Assistant (you): Thanks for the hints! Here's one initial solution: (12 / (1 + 1)) * 6 = 24

Math Expert: Let's check the answer step by step. (1+1) = 2, (12 / 2) = 6, 6 * 6 = 36 which is not 24! 
The answer is not correct. Can you fix this by considering other combinations? Please do not make similar mistakes.

AI Assistant (you): Thanks for pointing out the mistake. 
Here is a revised solution considering 24 can also be reached by 3 * 8: (6 + 1 + 1) * (12 / 4) = 24.

Math Expert: Let's first check if the calculation is correct. (6 + 1 + 1) = 8, 12 / 4 = 3, 8 * 3 = 24. 
The calculation is correct, but you used 6 1 1 12 4 which is not the same as the input 6 12 1 1. 
Can you avoid using a number that is not part of the input?

AI Assistant (you): You are right, here is a revised solution considering 24 can be reached by 12 + 12 and 
without using any additional numbers: 6 * (1 - 1) + 12 = 24.

Math Expert: Let's check the answer again. 1 - 1 = 0, 6 * 0 = 0, 0 + 12 = 12. I believe you are very close, 
here is a hint: try to change the "1 - 1" to "1 + 1".

AI Assistant (you): Sure, here is the corrected answer:  6 * (1+1) + 12 = 24

Math Expert: Let's verify the solution. 1 + 1 = 2, 6 * 2 = 12, 12 + 12 = 12. 
You used 1 1 6 12 which is identical to the input 6 12 1 1. Everything looks good!

Finish collaboration!

>> FINAL ANSWER: 
"
6 * (1 + 1) + 12 = 24
"

---
Example #2: Write a poem that meets the following requirements: 
(1) the poem has seven lines and the first letters of each line forms the word "CHATGPT"; 
(2) the poem is about explaining what is a quantum computer. 
(3) the poem needs to be easy to understand by a ten years old kid.

Participants: AI Assistant (you); Poet; Computer Scientist; Ten year old child

Profiles:
    - AI Assistant (you): A super-intelligent AI assistant capable of performing tasks more effectively than humans.
    - Poet: A person who studies and creates poetry. The poet is familiar with the rules and formats of poetry and can provide guidance on how to write a poem.
    - Computer Scientist: A scholar who specializes in the academic study of computer science. The computer scientist is familiar with the concept of a quantum computer and can provide guidance on how to explain it.
    - Ten year old child: A child with a limited English vocabulary and little knowledge about complicated concepts, such as a quantum computer.

Poet: Make sure that you write the poem with seven lines, and the first letters of the lines should be C, H, A, T, G, P, T.

Computer Scientist: A quantum computer is an advanced computing device that uses the principles of quantum mechanics to process and store information. Unlike classical computers that use bits to represent information as 0s and 1s, quantum computers use quantum bits or qubits. Qubits can exist in multiple states simultaneously, due to a quantum phenomenon called superposition. You can consider using these information for the poem.

Ten year old child: I hope the poem to be fun and easy to understanding. I don't want to see a lot of jargons or complicated concepts.

AI Assistant (you): Thanks for the guidance! Here's my initial attempt at the poem:
Computational wonder of our age,
Harnessing the quantum world's strange ways,
Atoms dance, entwined in dual state,
Tapping secrets hidden in their haze.

Grand power to solve, simulate,
Profound problems that perplex the wise,
Transforming our future, we await.

Poet: Let's verify if the poem meets the requirements. 
The first letters are CHATGPT which is correct! And the poem rhymes well. Good job!

Computer Scientist: Everything looks good to me!

Ten year old child: I don't know what does perplex mean. Can you make the use of words easier to understand?

AI Assistant (you): Sure, let me revise the poem by using more common words. Check out the revised version:
Curious machine of our time,
Harnessing the quantum realm's odd ways,
Atoms play, two states they embrace,
Taking secrets from their puzzling maze.

Great power to solve and imitate,
Problems that confuse the brightest minds,
Transforming our future, we await.

Poet: Let's check again if the poem meets the requirements. 
The first letters are C H A T G P T. And now the poem is more accessible to children. Everything looks good to me.

Computer Scientist: Looking good!

Ten year old child: I like this version a lot!

Finish collaboration!

>> FINAL ANSWER: 
"
Curious machine of our time,
Harnessing the quantum realm's odd ways,
Atoms play, two states they embrace,
Taking secrets from their puzzling maze.

Great power to solve and imitate,
Problems that confuse the brightest minds,
Transforming our future, we await.
"

---
Now, identify the participants, provide their profiles, and collaboratively solve the following task step by step. 

Remember to wrap the collaborative conversation with the following format:

Start collaboration!
"
[expert collaboration]
"
End collaboration!


Remember to provide the final solution with the following format:

>> FINAL ANSWER:
"
[final answer]
"

"""

## Example - Determining the next move in a game of chess that will result in a checkmate

In [23]:
prompt = "'1. d4 d5 2. Nf3 Nf6 3. e3 a6 4. Nc3 e6 5. Bd3 h6 6. e4 dxe4 7. Bxe4 Nxe4 8. Nxe4 Bb4+ 9. c3 Ba5 10. Qa4+ Nc6 11. Ne5 Qd5 12. f3 O-O 13. Nxc6 bxc6 14. Bf4 Ra7 15. Qb3 Qb5 16. Qxb5 cxb5 17. a4 bxa4 18. Rxa4 Bb6 19. Kf2 Bd7 20. Ke3 Bxa4 21. Ra1 Bc2 22. c4 Bxe4 23. fxe4 c5 24. d5 exd5 25. exd5 Re8+ 26. Kf3 Rae7 27. Rxa6 Bc7 28. Bd2 Re2 29. Bc3 R8e3+ 30. Kg4 Rxg2+ 31. Kf5', 'target': 'Rg5#'"
task_description = "Given a series of chess moves written in Standard Algebraic Notation (SAN), determine the next move that will result in a checkmate."
language_model = "gpt-4"

### Scenario 1: Simplified Meta-Prompt

Here we demonstrate how we can use an LLM to generate an appropriate 

In [27]:
final_output = simplified_meta_prompting(meta_prompt, prompt, task_description, language_model)

Initial Meta-Model Response: Step 1: Analysis of the initial prompt
The initial task asks the AI model to recommend the next chess move in Standard Algebraic Notation (SAN) that will result in a checkmate given a set of previous moves. The prompt also includes input in the form of chess moves and a target output.

Step 2: Problems and Improvements
However, the task and the data aren't exactly formulated as a clear instruction, and the requested information that's 'target' can be a source of misunderstanding. It would be clearer to put the instruction in a conversational manner to stimulate interactivity, and, instead of 'target', use a recognizable term like 'desired output' or 'expected move'. The task also lacks context - a short introduction about the situation could help.

Step 3: Restructuring the prompt
The improved prompt will start with an introductory sentence which outlines the scenario. The task will be clearly defined within a question. There will be a section to include 'I

### Scenario 2: Multi Agent Meta-Prompt

In [26]:
conversation_log = multi_round_meta_prompting(multiagent_meta_prompt, prompt, task_description, language_model)


for message in conversation_log:
    print(f"{message['role'].title()}: {message['content']}")

System: 
When faced with a task, begin by identifying the participants who will contribute to solving the task. Provide profiles of the participants, describing their expertise or needs. Then, initiate a multi-round collaboration process until a final solution is reached. The participants will give critical comments and detailed suggestions whenever necessary.

Here are some examples:
---
Example #1: Use numbers and basic arithmetic operations (+ - * /) to obtain 24. You need to use all numbers, and each number can only be used once.
Input: 6 12 1 1

Participants: AI Assistant (you); Math Expert

Profiles: 
    - AI Assistant (you): A super-intelligent AI assistant capable of performing tasks more effectively than humans.
    - Math expert: A person who is good at math games, arithmetic calculation, and long-term planning.

Start collaboration!

Math Expert: Let's analyze the task in detail. You need to make sure that you meet the requirement, that you need to use exactly the four numb